In [3]:
import time
import datetime
from mkndaq.inst.neph import NEPH
import schedule
import threading

cfg = {'ne300': {
                'type': 'NE300',
                'serial_number': '23-0690',
                'serial_id': 0,
                'protocol': 'acoem',
                'socket': {
                    'host': '192.168.0.50',
                    'port': 32783,
                    'timeout': 10,
                },
                'datalog': {
                    'parameters': [3635000, 3525000, 3450000, 3635090, 3525090, 3450090, 
                                   2635000, 2525000, 2450000, 2635090, 2525090, 2450090, 
                                   8635000, 8525000, 8450000, 8635090, 8525090, 8450090, 
                                   7635000, 7525000, 7450000, 7635090, 7525090, 7450090, 
                                   5001, 5002, 5003, 5004, 5005, 
                                   13635000, 13525000, 13450000, 
                                   15635000, 15525000, 15450000, 
                                   11635000, 11525000, 11450000, 11635090, 11525090, 11450090, 
                                   6007, 6008, 6001, 6002, 6003, 
                                   5011, 5012, 5014, 4035],
                    'parameters': [2635000, 2525000, 2450000, 2635090, 2525090, 2450090, 
                                #    3635000, 3525000, 3450000, 3635090, 3525090, 3450090, 
                                #    12635000, 12525000, 12450000, 12635090, 12525090, 12450090, 
                                #    13635000, 13525000, 13450000, 13635090, 13525090, 13450090, 
                                #    14635000, 14525000, 14450000, 14635090, 14525090, 14450090,
                                #    15635000, 15525000, 15450000, 15635090, 15525090, 15450090,
                                #    16635000, 16525000, 16450000, 16635090, 16525090, 16450090,
                                   5001, 5002, 5003, 5004, 5005, 5006, 5010, 
                                   26635000, 26525000, 26450000, 
                                   13525000, 
                                   15635000, 15525000, 15450000, 
                                   11635000, 11525000, 11450000, 11635090, 11525090, 11450090, 
                                   6007, 6008, 6001, 6002, 6003, 
                                   6635000, 6525000, 6450000, 6635090, 6525090, 6450090, 
                                   5011, 5012, 5014, 4035],
                    'interval': 60, 
                },
                'get_data_interval': 3, # minutes. logger retrieval interval
                'zero_span_check_interval': 13,
                'zero_check_duration': 5,
                'span_check_duration': 3,
                'staging_zip': True,  
                'verbosity': 2,  # 0: silent, 1: medium, 2: full          
            },
            'reporting_interval': 10,
            'logs': 'mkndaq/logs',
            'data': 'mkndaq/data',
            'staging': {
                'path': 'mkndaq/staging',
            },
}
ne300 = NEPH('ne300', cfg, verbosity=0)
# # Initialize NEPH (name: ne300  S/N: 23-0690)
#   Instrument identified itself as '{'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}'.

# Initialize NEPH (name: ne300  S/N: 23-0690)
  - Instrument identified itself as '{'id': 'ACOEM Aurora NE-300 Sub-Type: 0 Range: 0 Build: 158 Branch: 300'}'.
  - Instrument current operation: ambient.
  - Logging config set to: ["'set_datalog_config' not yet implemented."].
  - Logging config reported by instrument: [3635000, 3525000, 3450000, 3635090, 3525090, 3450090, 2635000, 2525000, 2450000, 2635090, 2525090, 2450090, 8635000, 8525000, 8450000, 8635090, 8525090, 8450090, 7635000, 7525000, 7450000, 7635090, 7525090, 7450090, 5001, 5002, 5003, 5004, 5005, 13635000, 13525000, 13450000, 15635000, 15525000, 15450000, 11635000, 11525000, 11450000, 11635090, 11525090, 11450090, 6007, 6008, 6001, 6002, 6003, 5011, 5012, 5014, 4035].
  - Datalog interval set to 60 seconds.


In [2]:
import logging

def run_threaded(job_func):
    """Set up threading and start job.

    Args:
        job_func ([type]): [description]
    """
    job_thread = threading.Thread(target=job_func)
    job_thread.start()

# limit logging from schedule
logging.getLogger('schedule').setLevel(logging.CRITICAL)

# align start with a 10' timestamp
while int(time.time()) % 10 > 0:
    time.sleep(0.1)

fetch = 20
schedule.every(fetch).seconds.do(ne300.print_ssp_bssp)
schedule.every(cfg['ne300']['get_data_interval']).minutes.at(':10').do(run_threaded, ne300.get_new_data)
schedule.every(cfg['ne300']['zero_span_check_interval']).minutes.at(':00').do(run_threaded, ne300.do_zero_span_check)

while True:
    schedule.run_pending()
    time.sleep(1)

2024-06-01 06:47:50 [ne300] ssp|bssp (Mm-1) r: -0.2281|-0.7473 g: 0.8784|0.1248 b: 1.0640|0.2755
2024-06-01 06:48:10 [ne300] ssp|bssp (Mm-1) r: -0.2322|-0.7385 g: 0.8229|0.1447 b: 1.0852|0.2613
2024-06-01 06:48:30 [ne300] ssp|bssp (Mm-1) r: -0.2371|-0.7407 g: 0.9008|0.1385 b: 1.0939|0.2628
2024-06-01 06:48:50 [ne300] ssp|bssp (Mm-1) r: -0.2003|-0.7385 g: 1.0544|0.1168 b: 1.0943|0.3000
2024-06-01 06:49:10 [ne300] ssp|bssp (Mm-1) r: -0.1990|-0.7405 g: 1.1366|0.0993 b: 1.0835|0.2608
2024-06-01 06:49:30 [ne300] ssp|bssp (Mm-1) r: -0.2047|-0.7414 g: 1.1307|0.1532 b: 1.0495|0.2462
2024-06-01 06:49:50 [ne300] ssp|bssp (Mm-1) r: -0.2264|-0.7499 g: 0.9918|0.1668 b: 0.9941|0.2152
2024-06-01 06:50:10 .get_new_data (name=ne300, save=True)
2024-06-01 06:50:10 [ne300] ssp|bssp (Mm-1) r: -0.2379|-0.7532 g: 0.9042|0.1539 b: 1.0173|0.2031
2024-06-01 06:50:30 [ne300] ssp|bssp (Mm-1) r: -0.2139|-0.7566 g: 0.8731|0.1739 b: 0.9704|0.2186
2024-06-01 06:50:50 [ne300] ssp|bssp (Mm-1) r: -0.2206|-0.7583 g: 0.8

KeyboardInterrupt: 

In [ ]:
res = ne300.get_id()
print(res)

In [ ]:
# mimick VI099 response
ne300.get_current_data(strict=True, verbosity=0)

In [ ]:
ne300.get_current_data(verbosity=0)

In [ ]:
ne300.get_values([1635000, 1525000], verbosity=2)

In [ ]:
# retrieve VI099 command parameters
ne300.get_values(parameters=[1, 1635000,1525000,1450000,1635090,1525090,1450090,5001,5004,5003,5002,4036,4035])

In [ ]:
ne300.get_current_data(add_params=[1001, 1002, 1003, 1004, 1005, 2001, 2002])

In [ ]:
# not implemented
ne300.get_new_data()

In [ ]:
# not implemented
ne300.get_all_data()

In [ ]:
ne300.get_instr_type()

In [ ]:
ne300.get_datetime()

In [ ]:
ne300.get_datalog_config()

In [ ]:
# cycle through operating states
# 0: ambient, 1: zero, 2: span
for i in [1, 2, 0]:
    print(f"current: {ne300.get_current_operation()} > set: {i} ...")
    print(f"done: {ne300.set_current_operation(state=i)} okay")
    time.sleep(3)

In [ ]:
from datetime import datetime, timezone, timedelta
end = datetime.now(timezone.utc)
start = end - timedelta(minutes=10)
print(f"start: {start}\nend  : {end}")
data = ne300.get_logged_data(start=start, end=end, verbosity=1)
# for i in range(len(data)):
    # print(data[i])

In [ ]:
# repeat packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,1]))
response = ne300.tcpip_comm(message=message)
print(response)
display(ne300._acoem_decode_logged_data(response=response))

In [ ]:
# next packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,0]))
response = ne300.tcpip_comm(message=message)
response